In [1]:
import pandas as pd
import numpy as np
import requests
import os
import matplotlib.pyplot as plt

# Download the Dataset

In [3]:
os.getcwd()

'/Users/mattwilliams/Documents/RealPython/PandasDataScience'

In [7]:
def download_dataset(url, fname, ext):
    target_path = f'{fname}.{ext}'
    p_dir = os.getcwd()
    dataset = requests.get(url)
    response = dataset.raise_for_status()
    print(f'Response: {response}')
    with open(target_path, 'wb') as f:
        f.write(dataset.content)
    print(f'Download of {target_path} completed successfully at {p_dir}.')

In [9]:
url = 'https://theunitedstates.io/congress-legislators/legislators-historical.csv'
download_dataset(url, 'legislators-historical', 'csv')

Response: None
Download of legislators-historical.csv completed successfully at /Users/mattwilliams/Documents/RealPython/PandasDataScience.


In [13]:
dtypes = {
    'first_name': 'category',
    'gender': 'category',
    'type': 'category',
    'state': 'category',
    'party': 'category'
}

df = pd.read_csv('legislators-historical.csv',
                dtype=dtypes,
                usecols=list(dtypes) + ['birthday', 'last_name'],
                parse_dates=['birthday'])

In [14]:
df.tail()

,last_name,first_name,birthday,gender,type,state,party
11977,Isakson,John,1944-12-28,M,sen,GA,Republican
11978,Hunter,Duncan,1976-12-07,M,rep,CA,Republican
11979,Meadows,Mark,1959-07-28,M,rep,NC,Republican
11980,Ratcliffe,John,1965-10-20,M,rep,TX,Republican
11981,Lewis,John,1940-02-21,M,rep,GA,Democrat


In [15]:
df.dtypes

last_name             object
first_name          category
birthday      datetime64[ns]
gender              category
type                category
state               category
party               category
dtype: object

# The 'Hello World' of GroupBy

What is the count of congressional members, on a state-by-state basis, over the entire history of the dataset?

In [16]:
n_by_state = df.groupby('state')['last_name'].count()
n_by_state

state
AK      16
AL     206
AR     117
AS       2
AZ      48
CA     363
CO      90
CT     240
DC       2
DE      97
DK       9
FL     155
GA     311
GU       4
HI      23
IA     202
ID      59
IL     486
IN     341
KS     141
KY     373
LA     197
MA     426
MD     305
ME     175
MI     294
MN     160
MO     333
MS     155
MT      52
NC     354
ND      44
NE     127
NH     181
NJ     359
NM      54
NV      56
NY    1462
OH     674
OK      92
OL       2
OR      89
PA    1053
PI      13
PR      19
RI     107
SC     251
SD      51
TN     299
TX     257
UT      53
VA     432
VI       4
VT     115
WA      95
WI     197
WV     120
WY      40
Name: last_name, dtype: int64

We can actually go even farther than this and group by state and gender.

In [18]:
df.groupby(['state','gender'])['last_name'].count()

state  gender
AK     F           0
       M          16
AL     F           3
       M         203
AR     F           5
                ... 
WI     M         197
WV     F           1
       M         119
WY     F           2
       M          38
Name: last_name, Length: 116, dtype: int64

This is a MultiIndex DataFrame. We can set the keyword ```group_by(as_index=False)``` in order to push the grouped-on column back into a regular column.

In [21]:
n_by_state_gender = df.groupby(['state','gender'])['last_name'].count()
n_by_state_gender.index[:5]

MultiIndex([('AK', 'F'),
            ('AK', 'M'),
            ('AL', 'F'),
            ('AL', 'M'),
            ('AR', 'F')],
           names=['state', 'gender'])

In [20]:
df.groupby(['state','gender'], as_index=False)['last_name'].count()

,state,gender,last_name
0,AK,F,NaN
1,AK,M,16.0
2,AL,F,3.0
3,AL,M,203.0
4,AR,F,5.0
...,...,...,...
111,WI,M,197.0
112,WV,F,1.0
113,WV,M,119.0
114,WY,F,2.0


We now have a DataFrame with three columns and a RangeIndex, versus a MultiIndex and a Series.

# How GroupBy Works

In [22]:
by_state = df.groupby('state')
print(by_state)

In [23]:
by_state.__str__()

'<pandas.core.groupby.generic.DataFrameGroupBy object at 0x7fc63dab3750>'

This doesn't give us much information. But, we can iterate over it to try and gain some insight.

In [24]:
for state, frame in by_state:
    print(f'First 2 entries for {state}')
    print('------------------------')
    print(frame.head(2), end='\n\n')

First 2 entries for AK
------------------------
     last_name first_name   birthday gender type state        party
6618    Waskey      Frank 1875-04-20      M  rep    AK     Democrat
6646      Cale     Thomas 1848-09-17      M  rep    AK  Independent

First 2 entries for AL
------------------------
    last_name first_name   birthday gender type state       party
911   Crowell       John 1780-09-18      M  rep    AL  Republican
990    Walker       John 1783-08-12      M  sen    AL  Republican

First 2 entries for AR
------------------------
     last_name first_name   birthday gender type state party
1000     Bates      James 1788-08-25      M  rep    AR   NaN
1278    Conway      Henry 1793-03-18      M  rep    AR   NaN

First 2 entries for AS
------------------------
          last_name first_name   birthday gender type state     party
10796         Sunia       Fofó 1937-03-13      M  rep    AS  Democrat
11753  Faleomavaega        Eni 1943-08-15      M  rep    AS  Democrat

First 2 e

The ```group_by()``` method first split dataset up by state. ```by_state``` is a dictionary with states as keys.

In [28]:
by_state.groups['PA']

Int64Index([    4,    19,    21,    27,    38,    57,    69,    76,    84,
               88,
            ...
            11840, 11864, 11873, 11875, 11885, 11889, 11930, 11943, 11957,
            11971],
           dtype='int64', length=1053)

In [29]:
by_state.get_group('PA')

,last_name,first_name,birthday,gender,type,state,party
4,Clymer,George,1739-03-16,M,rep,PA,NaN
19,Maclay,William,1737-07-20,M,sen,PA,Anti-Administration
21,Morris,Robert,1734-01-20,M,sen,PA,Pro-Administration
27,Wynkoop,Henry,1737-03-02,M,rep,PA,NaN
38,Jacobs,Israel,1726-06-09,M,rep,PA,NaN
...,...,...,...,...,...,...,...
11889,Brady,Robert,1945-04-07,M,rep,PA,Democrat
11930,Shuster,Bill,1961-01-10,M,rep,PA,Republican
11943,Rothfus,Keith,1962-04-25,M,rep,PA,Republican
11957,Costello,Ryan,1976-09-07,M,rep,PA,Republican


How does ```group_by()``` apply the operation. Since the object contains a bunch of mini dataframes, you can iterate through each, which will return the state and the frame.

In [31]:
state, frame = next(iter(by_state))
state

'AK'

In [38]:
frame.head(3)

,last_name,first_name,birthday,gender,type,state,party
6618,Waskey,Frank,1875-04-20,M,rep,AK,Democrat
6646,Cale,Thomas,1848-09-17,M,rep,AK,Independent
7441,Grigsby,George,1874-12-02,M,rep,AK,NaN


In [47]:
state, frame = next(iter(by_state))
state

'AK'

In [48]:
frame.head(3)

,last_name,first_name,birthday,gender,type,state,party
6618,Waskey,Frank,1875-04-20,M,rep,AK,Democrat
6646,Cale,Thomas,1848-09-17,M,rep,AK,Independent
7441,Grigsby,George,1874-12-02,M,rep,AK,NaN


In [65]:
state_iter = iter(by_state)

<generator object BaseGrouper.get_iterator at 0x7fc63d8ac650>

In [66]:
next(state_iter)

('AK',
         last_name first_name   birthday gender type state        party
 6618       Waskey      Frank 1875-04-20      M  rep    AK     Democrat
 6646         Cale     Thomas 1848-09-17      M  rep    AK  Independent
 7441      Grigsby     George 1874-12-02      M  rep    AK          NaN
 7500       Sulzer    Charles 1879-02-24      M  rep    AK          NaN
 8038   Sutherland     Daniel 1869-04-17      M  rep    AK   Republican
 8235   Wickersham      James 1857-08-24      M  rep    AK   Republican
 8876       Dimond    Anthony 1881-11-30      M  rep    AK     Democrat
 9818     Gruening     Ernest 1887-02-06      M  sen    AK     Democrat
 9950       Rivers      Ralph 1903-05-23      M  rep    AK     Democrat
 9984     Bartlett     Edward 1904-04-20      M  sen    AK     Democrat
 10081     Pollock     Howard 1920-04-11      M  rep    AK   Republican
 10107      Begich   Nicholas 1932-04-06      M  rep    AK     Democrat
 10324      Gravel    Maurice 1930-05-13      M  sen    A

In [68]:
state, frame = next(state_iter)

In [69]:
state

'AL'

# Air Quality Dataset

In [71]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip'
download_dataset(url, 'air_quality', 'zip')

Response: None
Download of air_quality.zip completed successfully at /Users/mattwilliams/Documents/RealPython/PandasDataScience.


In [73]:
import zipfile

In [74]:
with zipfile.ZipFile('air_quality.zip') as zipobj:
    zipobj.extractall()

In [119]:
df = pd.read_csv(
    "AirQualityUCI.csv",
    header=0,
    sep = ';',
    decimal = ',',
    parse_dates=[["Date", "Time"]],
    na_values=[-200],
    usecols=["Date", "Time", "CO(GT)", "T", "RH", "AH"],
    skip_blank_lines=True
).rename(
    columns={
        "CO(GT)": "co",
        "Date_Time": "tstamp",
        "T": "temp_c",
        "RH": "rel_hum",
        "AH": "abs_hum",
    }
)

In [120]:
df.dropna(inplace=True, axis=0)

In [121]:
df.dtypes

tstamp      object
co         float64
temp_c     float64
rel_hum    float64
abs_hum    float64
dtype: object

In [140]:
df['tstamp'] = df['tstamp'].replace('\.',':', regex=True)

In [141]:
df['tstamp'] = pd.to_datetime(df['tstamp'])

In [144]:
df = df.set_index('tstamp')

In [146]:
df.head()

,co,temp_c,rel_hum,abs_hum
tstamp,,,,
2004-10-03 18:00:00,2.6,13.6,48.9,0.7578
2004-10-03 19:00:00,2.0,13.3,47.7,0.7255
2004-10-03 20:00:00,2.2,11.9,54.0,0.7502
2004-10-03 21:00:00,2.2,11.0,60.0,0.7867
2004-10-03 22:00:00,1.6,11.2,59.6,0.7888


In [147]:
df.index.min()

Timestamp('2004-01-04 00:00:00')

In [148]:
df.index.max()

Timestamp('2005-12-03 23:00:00')

## Group on Derived Arrays

In [150]:
day_names = df.index.day_name()

In [152]:
day_names[:10]

Index(['Sunday', 'Sunday', 'Sunday', 'Sunday', 'Sunday', 'Sunday', 'Wednesday',
       'Wednesday', 'Wednesday', 'Wednesday'],
      dtype='object', name='tstamp')

In [153]:
df.groupby(day_names)['co'].mean()

tstamp
Friday       2.427715
Monday       2.068280
Saturday     1.948936
Sunday       1.644991
Thursday     2.238169
Tuesday      2.302899
Wednesday    2.327192
Name: co, dtype: float64

In [154]:
hr = df.index.hour

In [155]:
df.groupby([day_names, hr])['co'].mean().rename_axis(['dow', 'hr'])

dow        hr
Friday     0     1.815217
           1     1.448889
           2     1.071111
           3     0.848889
           4     0.732258
                   ...   
Wednesday  19    4.422222
           20    3.962222
           21    2.797826
           22    2.097826
           23    1.944444
Name: co, Length: 168, dtype: float64

In [156]:
bins = pd.cut(df['temp_c'], bins=3, labels=('cool', 'warm', 'hot'))

In [157]:
df[['rel_hum', 'abs_hum']].groupby(bins).agg(['mean', 'median'])

rel_hum          abs_hum        
             mean median      mean  median
temp_c                                    
cool    57.450057   58.9  0.662018  0.6523
warm    48.444528   48.3  1.151161  1.1174
hot     24.814408   23.7  1.284418  1.2515

In [158]:
type(bins)

pandas.core.series.Series

In [159]:
bins.head()

tstamp
2004-10-03 18:00:00    cool
2004-10-03 19:00:00    cool
2004-10-03 20:00:00    cool
2004-10-03 21:00:00    cool
2004-10-03 22:00:00    cool
Name: temp_c, dtype: category
Categories (3, object): [cool < warm < hot]

## Resampling

If we wanted to aggregate by quarter, we would use ```groupby``` to accomplish this:

In [160]:
df.groupby([df.index.year, df.index.quarter])['co'].agg(['max','min']).rename_axis(['year', 'quarter'])

max  min
year quarter           
2004 1         9.4  0.1
     2         8.7  0.1
     3         7.5  0.1
     4        11.9  0.1
2005 1         8.7  0.1
     2         6.1  0.4
     3         6.0  0.4
     4         5.9  0.1

Another way to do this is by _resampling_.

In [161]:
df.resample('Q')['co'].agg(['max','min'])

,max,min
tstamp,,
2004-03-31,9.4,0.1
2004-06-30,8.7,0.1
2004-09-30,7.5,0.1
2004-12-31,11.9,0.1
2005-03-31,8.7,0.1
2005-06-30,6.1,0.4
2005-09-30,6.0,0.4
2005-12-31,5.9,0.1
